In [ ]:
# pip install xgboost

In [1]:
from sklearn.metrics import accuracy_score, mean_absolute_error, precision_score, recall_score, f1_score
import xgboost
from xgboost import XGBClassifier, cv

import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import pandas as pd
import numpy as np

import datetime
from datetime import datetime as dt

import requests
import json
import csv

import warnings
warnings.filterwarnings('ignore')

In [26]:
def engineering(n, d):
    # 성별 인코딩
    d['gender'] = d['gender'].apply(lambda x:0 if x == 'M' else 1)
    # 날짜 변환
    d['date'] = pd.to_datetime(d['date'])
    # 스트레스(target)
    d['pmStress'] = d['pmStress'] - 1
    
    # 감정변화비율 = 오후감정/오전감정
    d['emotionChangeRate'] = d['pmEmotion'] / d['amEmotion']
    # 긍정변화평균 = emotion1~7의 평균
    d['positiveMean'] = d.filter(regex='Positive').mean(axis=1)
    # 긴장변화평균 = emotion1~7의 평균
    d['tensionnMean'] = d.filter(regex='Tension').mean(axis=1)
    
    # 긍정비율 = Positive5~7 / 1~3 > 4는 중앙값으로 진행하지 않음
    d['positiveRate'] = d[['emotionPositive5', 'emotionPositive6', 'emotionPositive7']].sum(axis=1) / d[['emotionPositive1', 'emotionPositive2', 'emotionPositive3']].sum(axis=1)
    # 긴장비율 = Tension5~7 / 1~3 > 4는 중앙값으로 진행하지 않음
    d['tensionRate'] = d[['emotionTension5', 'emotionTension6', 'emotionTension7']].sum(axis=1) / d[['emotionTension1', 'emotionTension2', 'emotionTension3']].sum(axis=1)
    
    # 긍정감정의 임시 테이블
    pos_temp = d.filter(regex='Positive')
    # 가장 높은 긍정감정의 숫자(1~7)
    d['topPositive'] = pos_temp.idxmax(axis=1).apply(lambda x:int(x[-1]))
    # 가장 낮은 긍정감정의 숫자(1~7)
    d['botPositive'] = pos_temp.idxmin(axis=1).apply(lambda x:int(x[-1]))
    
    # 긍정 수치 1~7 * count 수
    for p, c in enumerate(pos_temp.columns):
        d.loc[:, [c]] = d.loc[:, [c]] * (p+1)
        
    del pos_temp
    
    # 긴장감정의 임시 테이블
    ten_temp = d.filter(regex='Tension')
    # 가장 높은 긴장감정의 숫자(1~7)
    d['topTension'] = ten_temp.idxmax(axis=1).apply(lambda x:int(x[-1]))
    # 가장 낮은 긴장감정의 숫자(1~7)
    d['botTension'] = ten_temp.idxmin(axis=1).apply(lambda x:int(x[-1]))
    
    # 긴장 수치 1~7 * count 수
    for t, c in enumerate(ten_temp.columns):
        d.loc[:, [c]] = d.loc[:, [c]] * (t+1)
    
    del ten_temp
    
    # 수치 반영 긍정 평균
    d['positiveWMean'] = d.filter(regex='Positive').mean(axis=1)
    # 수치 반영 긴장 평균
    d['tensionWMean'] = d.filter(regex='Tension').mean(axis=1)
    
    # 수치 반영 긍정 비율
    d['positiveWRate'] = d[['emotionPositive5', 'emotionPositive6', 'emotionPositive7']].sum(axis=1) / d[['emotionPositive1', 'emotionPositive2', 'emotionPositive3']].sum(axis=1)
    # 수치 반영 긴장 비율
    d['tensionWRate'] = d[['emotionTension5', 'emotionTension6', 'emotionTension7']].sum(axis=1) / d[['emotionTension1', 'emotionTension2', 'emotionTension3']].sum(axis=1)
    
    # 오전 컨디션에 긍정/긴장 반영 = 오전 컨디션 * (수치 반영 긍정 평균 + 수치 반영 긴장 평균) / 7 -> 수치가 1~7이니까 그냥 7로 나누어 봄ㅎㅎ..
    d['aCmET'] = round(d['amCondition'] * (d['positiveWMean'] + d['tensionWMean']) / 7)
    # 오전 감정에 긍정/긴장 반영 = 오전 감정 * (수치 반영 긍정 평균 + 수치 반영 긴장 평균) / 7 -> 이것도
    d['aEmET'] = round(d['amEmotion'] * (d['positiveWMean'] + d['tensionWMean']) / 7)
    # 오전 컨디션/감정에 긍정 긴장 반영한 것의 평균을 냄
    d['aCEmET'] = round((d['aCmET'] + d['aEmET']) / 2)
    
    # 오전 컨디션 긍정/긴장 반영에 오후 감정을 더한 뒤 평균을 냄
    d['aCmETpE'] = round((d['aCmET'] + d['pmEmotion']) / 2)
    # 오전 감정 긍정/긴장 반영에 오후 감정을 더한 뒤 평균을 냄
    d['aEmETpE'] = round((d['aEmET'] + d['pmEmotion']) / 2)
    # 오전 컨디션/감정 긍정/긴장 반영에 오후 감정을 더한 뒤 평균을 냄
    d['aCEmETpE'] = round((d['aCEmET'] + d['pmEmotion']) / 2)
    
    # 최고 수치를 반영한 오전 감정 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['aCtPT'] = round(d['amEmotion'] * (d['topPositive'] + d['topTension']) / 14)
    # 최고 수치를 반영한 오전 컨디션 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['aEtPT'] = round(d['amCondition'] * (d['topPositive'] + d['topTension']) / 14)
    # 최고 수치를 반영한 오후 감정 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['pEtPT'] = round(d['pmEmotion'] * (d['topPositive'] + d['topTension']) / 14)
    
    # 둘 중에 하나로 쓰거나 컬럼명 변경해서 모두 써도 됩니다.
    # (최고 수치를 반영한 오전 컨디션 + 최고 수치를 반영한 오전 감정 + 최고 수치를 반영한 오후 감정) / 3
    # (최고 수치를 반영한 오전 컨디션 + 최고 수치를 반영한 오전 감정 + 최고 수치를 반영한 오후 감정)
#     d['aCEpEtPTm'] = round((d['aCtPT'] + d['aEtPT'] + d['pEtPT']) / 3)
    d['aCEpEtPTm'] = (d['aCtPT'] + d['aEtPT'] + d['pEtPT'])
    
    # 긍정적인지 = 수치 반영 긍정 평균이 중앙값보다 크면 1 아니면 0
    d['positive'] = d['positiveWRate'].apply(lambda x:1 if x > d['positiveWRate'].median() else 0)
    # 부정적인지 = 수치 반영 긍정 평균이 중앙값보다 작으면 1 아니면 0
    d['negative'] = d['positiveWRate'].apply(lambda x:1 if x < d['positiveWRate'].median() else 0)
    
    # 긴장 상태인지 = 수치 반영 긴장 평균이 중앙값보다 크면 1 아니면 0
    d['aroused'] = d['tensionWRate'].apply(lambda x:1 if x > d['tensionWRate'].median() else 0)
    # 편안한 상태인지 = 수치 반영 긴장 평균이 중앙값보다 작으면 1 아니면 0
    d['relaxed'] = d['tensionWRate'].apply(lambda x:1 if x < d['tensionWRate'].median() else 0)
    
    # 활동 비율 = 자전거, 도보의 합 / 운송수단, 가만히 있기의 합
    d['activityRate'] = d[['on_bicycle', 'on_foot']].sum(axis=1) / d[['in_vehicle', 'still']].sum(axis=1)
    # 0으로 나누기된 것을 0으로 치환
    d.replace([np.inf, -np.inf], 0, inplace=True)
    d.dropna(inplace=True)
    # random_state는 잘 나오는 것으로 바꿔보세요! 제가 해본 것 : 1111, 2222, 3333, 4444, 5555, 6666, 7777, 8888, 9999, 1234
    d.sample(n=d.shape[0], random_state=8888)
    
    return d

In [27]:
df1 = pd.read_csv(r'C:\Users\rian\Documents\2023\02.nonmun\02.result\lifelog_data_2018.csv')
df2 = pd.read_csv(r'C:\Users\rian\Documents\2023\02.nonmun\02.result\lifelog_data_2019.csv')
df3 = pd.read_csv(r'C:\Users\rian\Documents\2023\02.nonmun\02.result\lifelog_data_2020.csv')
for n, d in enumerate([df1, df2, df3]):
    globals() [f'df{n+1}'] = engineering(n, d.copy())

In [28]:
df = pd.concat([df1, df2, df3], axis=0)

In [29]:
df = df[[c for c in df.columns if c not in ['pmStress']] + ['pmStress']]

In [30]:
train = pd.DataFrame()
valid = pd.DataFrame()
test = pd.DataFrame()
for i in range(5):
    train = pd.concat([train, df.loc[df['pmStress'] == np.unique(df['pmStress'])[i]][:round(len(df) / 5 * 0.7)]], axis=0)
    valid = pd.concat([valid, df.loc[df['pmStress'] == np.unique(df['pmStress'])[i]][round(len(df) / 5 * 0.7):-round(len(df) / 5 * 0.1)]], axis=0)
    test = pd.concat([test, df.loc[df['pmStress'] == np.unique(df['pmStress'])[i]][-round(len(df) / 5 * 0.1):]], axis=0)

    # 동 수로 뽑을 때
#     temp = df.loc[df['pmStress'] == np.unique(df['pmStress'])[i]][:151]
#     train = pd.concat([train, temp[:round(151 * 0.7)]], axis=0)
#     valid = pd.concat([valid, temp[round(151 * 0.7):-round(151 * 0.1)]], axis=0)
#     test = pd.concat([test, temp[-round(151 * 0.1):]], axis=0)

train = train.sample(n=train.shape[0]) # , random_state=1234)
valid = valid.sample(n=valid.shape[0]) # , random_state=1234)
test = test.sample(n=test.shape[0]) # , random_state=1234)

In [31]:
df = pd.concat([train, test, valid], axis=0)
# sns.countplot(x='pmStress', data=df)

In [32]:
train_X = train.iloc[:, 2:-1].values
train_y = train.iloc[:, -1].values.reshape(-1, 1)
valid_X = valid.iloc[:, 2:-1].values
valid_y = valid.iloc[:, -1].values.reshape(-1, 1)
test_X = test.iloc[:, 2:-1].values
test_y = test.iloc[:, -1].values.reshape(-1, 1)

In [39]:
# learning rate
lr = 0.1
# max_depth
md = 4
# n_estimators
ne = 5
# subsample
ss = 0.9
# min_child_weight
mc = 0
# gamma
g = 0.5
# num_parallel_tree
pt = 10
# booster
b = 'gbtree'
# tree_method
tm = 'hist'
# eval_metric
em = 'mlogloss'
# objective
o = 'multi:softmax'
# lambda
l = 0.7
# alpha
a = 0.7
# coldample_bytree
cb = 0.2

params = {'learning_rate' : lr, 'max_depth': md, 'n_estimators': ne,
        'num_class' : 5, 'subsample': ss,
        'colsample_bytree': cb, 'min_child_weight': mc,
        'lambda': l, 'alpha': a,
        'gamma': g, 'num_parallel_tree' : pt,
        'booster': b, 'tree_method': tm,
        'eval_metric' : em, 'objective': o,
        'random_state' : 6666, 'verbosity': 0}

model = XGBClassifier(**params)

model.fit(train_X.copy(), train_y.copy(),
        eval_set=[(train_X.copy(), train_y.copy()),(valid_X.copy(), valid_y.copy())],
        callbacks=[xgboost.callback.EarlyStopping(rounds=200)],
        verbose=False)

pred_XGB = model.predict(test_X.copy())
accuracy = accuracy_score(test_y.copy(), pred_XGB.copy())
print(lr, md, ne, ss, mc, g, pt, b, tm, em, o, l, a, cb, '\naccuracy :', accuracy)

0.1 4 5 0.9 0 0.5 10 gbtree hist mlogloss multi:softmax 0.7 0.7 0.2 
accuracy : 0.580952380952381


In [ ]:
pred_XGB = pd.DataFrame(pred_XGB)
pred_XGB.columns  = ['XGB']
pred_XGB.to_csv('XGB.csv')